In [1]:
import pandas as pd
import numpy as np
import os 
from datetime import date
from pathlib import PurePath

In [2]:
path = PurePath(os.getcwd(),'ventas', 'Bajio 18.txt')
df = pd.read_table(path, delimiter = ';', encoding="iso-8859-1")
df

,Año,Mes_Numero,cli_cve,pto_Cve,pto_Nombre,CU,CF,CFC,TA,Ing_Neto_SA_SIEPS,Ing_Bruto_SIEPS
0,2018,10,1620004476,97290,POWERADE Moras 1L 6pk ...,21.133760,20.0,10.000000,120.0,1870.65,1870.65
1,2018,12,603904109,382,SIDRAL 2 LT 8 G PETNR ...,11.271340,4.0,4.000000,32.0,394.40,438.24
2,2018,9,620138527,97820,FUZE TEA DURAZNO 600 ML ...,1.902039,3.0,0.750000,18.0,136.50,195.00
3,2018,12,1601061935,99457,Botellin PET Durazno 237ml 6pk ...,1.000000,4.0,1.000000,24.0,136.00,136.00
4,2018,10,604205525,99458,Botellin PET Guayaba 237ml 6pk ...,2.000000,8.0,2.000000,48.0,264.00,264.00
...,...,...,...,...,...,...,...,...,...,...,...
17332250,2018,12,1605015665,379,Fanta Naranja 355 ml 12PK lata ...,150.000000,200.0,100.000000,2400.0,18063.40,20070.40
17332251,2018,11,603404014,844,FANTA FRESA 600 ML PETNR 12 PK PRECIADO ...,1.902042,1.5,0.750000,18.0,107.28,107.28
17332252,2018,10,610155023,97289,POWERADE Naranja 1L 6pk ...,4.226752,4.0,2.000000,24.0,374.12,374.12
17332253,2018,12,620152662,2715,CC sin azucar 500ml VR 24pk ...,8.453504,4.0,4.000000,96.0,0.00,0.00


In [3]:
ndf = df[['cli_cve','pto_Nombre','CU']].groupby(['cli_cve','pto_Nombre']).sum()
ndf = ndf.reset_index()
ndf['cli_cve'] = ndf['cli_cve'].apply(lambda x: 'Tienda No {}'.format(str(x)).strip())
ndf['pto_Nombre'] = ndf['pto_Nombre'].str.upper()
ndf['pto_Nombre'] = ndf['pto_Nombre'].apply(lambda x: x.strip())
ndf['CU'] = pd.to_numeric(ndf['CU'])
ndf= ndf.dropna()
ndf

,cli_cve,pto_Nombre,CU
0,Tienda No 220056285,ADES CHOCOLATE 200 ML 4PK,0.422676
1,Tienda No 220056285,CC 355ML NR PET 12PK,11.250000
2,Tienda No 220056285,CC 500ML NR PET 12PK,34.870704
3,Tienda No 220056285,CC COLA BOT 500ML R VID 1X24U (U ALIG),38.040768
4,Tienda No 220056285,CC CLASICA 355 ML 12PK LATA,4.500000
...,...,...,...
6930011,Tienda No 1980000512,V.FRUT PON CITR 600ML PETNR 12P,50.721040
6930012,Tienda No 1980000512,YOLI LIMON BOT 600L NR 12PK,20.288416
6930013,Tienda No 1980000565,CC 500ML NR PET 12PK,2.113376
6930014,Tienda No 1980000566,CC CLASICA 355 ML 12PK LATA,4.500000


In [4]:
ndf.columns = ['tienda','producto','cu']
ndf['category'] = ''
ndf.loc[(ndf['cu'] < ndf['cu'].quantile(.25)),'category'] = 'BAJA VENTA'
ndf.loc[(ndf['cu'] >= ndf['cu'].quantile(.25)) & (ndf['cu'] < ndf['cu'].quantile(.5)),'category'] = 'MEDIA VENTA'
ndf.loc[(ndf['cu'] >= ndf['cu'].quantile(.5)) & (ndf['cu'] < ndf['cu'].quantile(.75)),'category'] = 'ALTA VENTA'
ndf.loc[(ndf['cu'] >= ndf['cu'].quantile(.75)),'category'] = 'MUY ALTA VENTA'
ndf

,tienda,producto,cu,category
0,Tienda No 220056285,ADES CHOCOLATE 200 ML 4PK,0.422676,BAJA VENTA
1,Tienda No 220056285,CC 355ML NR PET 12PK,11.250000,MEDIA VENTA
2,Tienda No 220056285,CC 500ML NR PET 12PK,34.870704,ALTA VENTA
3,Tienda No 220056285,CC COLA BOT 500ML R VID 1X24U (U ALIG),38.040768,ALTA VENTA
4,Tienda No 220056285,CC CLASICA 355 ML 12PK LATA,4.500000,BAJA VENTA
...,...,...,...,...
6930011,Tienda No 1980000512,V.FRUT PON CITR 600ML PETNR 12P,50.721040,MUY ALTA VENTA
6930012,Tienda No 1980000512,YOLI LIMON BOT 600L NR 12PK,20.288416,ALTA VENTA
6930013,Tienda No 1980000565,CC 500ML NR PET 12PK,2.113376,BAJA VENTA
6930014,Tienda No 1980000566,CC CLASICA 355 ML 12PK LATA,4.500000,BAJA VENTA


In [5]:
ndf = ndf[ndf['tienda'].isin(ndf['tienda'].unique()[:100])]
ndf

,tienda,producto,cu,category
0,Tienda No 220056285,ADES CHOCOLATE 200 ML 4PK,0.422676,BAJA VENTA
1,Tienda No 220056285,CC 355ML NR PET 12PK,11.250000,MEDIA VENTA
2,Tienda No 220056285,CC 500ML NR PET 12PK,34.870704,ALTA VENTA
3,Tienda No 220056285,CC COLA BOT 500ML R VID 1X24U (U ALIG),38.040768,ALTA VENTA
4,Tienda No 220056285,CC CLASICA 355 ML 12PK LATA,4.500000,BAJA VENTA
...,...,...,...,...
4758,Tienda No 600100694,VF GUAYABA 355ML NRPET 12PK,94.500000,MUY ALTA VENTA
4759,Tienda No 600100694,VF MANGO BOT 600ML NR 12PK,35.504728,ALTA VENTA
4760,Tienda No 600100694,VLLEFRT GYBA BOT 600ML PT NR12P,124.266548,MUY ALTA VENTA
4761,Tienda No 600100694,VLLEFRU MZA BOT 600ML NR PET12P,53.257092,MUY ALTA VENTA


In [6]:
ndf.to_csv('tiendas.csv')

In [7]:
ndf = df[['Año','Mes_Numero','pto_Nombre','CU']].copy()
ndf['fecha'] = ndf.apply(lambda x : date(x['Año'],x['Mes_Numero'],1), axis=1)
ndf['pto_Nombre'] = ndf['pto_Nombre'].str.upper()
ndf['pto_Nombre'] = ndf['pto_Nombre'].apply(lambda x : x.strip())
del ndf['Mes_Numero']
del ndf['Año']
ndf

,pto_Nombre,CU,fecha
0,POWERADE MORAS 1L 6PK,21.133760,2018-10-01
1,SIDRAL 2 LT 8 G PETNR,11.271340,2018-12-01
2,FUZE TEA DURAZNO 600 ML,1.902039,2018-09-01
3,BOTELLIN PET DURAZNO 237ML 6PK,1.000000,2018-12-01
4,BOTELLIN PET GUAYABA 237ML 6PK,2.000000,2018-10-01
...,...,...,...
17332250,FANTA NARANJA 355 ML 12PK LATA,150.000000,2018-12-01
17332251,FANTA FRESA 600 ML PETNR 12 PK PRECIADO,1.902042,2018-11-01
17332252,POWERADE NARANJA 1L 6PK,4.226752,2018-10-01
17332253,CC SIN AZUCAR 500ML VR 24PK,8.453504,2018-12-01


In [8]:
products = pd.pivot_table(ndf, values='CU', index='pto_Nombre',
                    columns=['fecha'], aggfunc=np.sum, fill_value=0)
products = products.reset_index()
products

fecha,pto_Nombre,2018-09-01,2018-10-01,2018-11-01,2018-12-01
0,(CCO+CCSA) 355ML LATA 24PK NR+4TRINEO,0.000000,0.000000,0.000000,0.000000
1,ADES ALMENDRA 946ML 3PK,58.478238,2073.864224,1411.297798,1326.001542
2,ADES CHOCOLATE 200 ML 4PK,0.000000,0.000000,3837.334512,10576.163649
3,ADES COCO 946ML 3PK,39.985120,1637.520602,684.568742,550.456652
4,ADES FRUT DNO 200 ML 40 PK,56.356720,0.000000,0.000000,0.000000
...,...,...,...,...,...
567,YOLI LIMON BOT 600L NR 12PK,64557.103195,64719.305274,71112.584592,74152.145627
568,YOLI LIMON POSTMIX B-IN-B 9LT,12564.858624,15517.771584,19557.698880,24188.208768
569,YOLI LIMON V.RET. 1/2LT 24 PACK,12287.168064,13073.343936,13185.352864,14713.323712
570,YOLI LIMON VRET 355ML 24 PACK,2464.500000,2917.500000,3556.500000,5437.500000


In [9]:
columns = products.columns[1:]
tendencias = products.copy()

def get_trend_serie(value):
    values = []
    for id_column in range(1,len(columns),1):
        values.append(value[columns[id_column]]-value[columns[id_column-1]])
    return values

tendencias['tendencia'] = products.apply(lambda x: get_trend_serie(x),axis=1)


for column in columns:
    del tendencias[column]
    
def get_type_tendece(val):
    val = np.average(val)
    if val > 0:
        return 'ALCISTA'
    elif val == 0:
        return 'ESTABLE'
    elif val < 0:
        return 'BAJISTA'
    else:
        return ""
    
tendencias['tendencia_type'] = tendencias['tendencia'].apply(lambda x: get_type_tendece(x))
tendencias

fecha,pto_Nombre,tendencia,tendencia_type
0,(CCO+CCSA) 355ML LATA 24PK NR+4TRINEO,"[0.0, 0.0, 0.0]",ESTABLE
1,ADES ALMENDRA 946ML 3PK,"[2015.385986, -662.5664259999999, -85.29625600...",ALCISTA
2,ADES CHOCOLATE 200 ML 4PK,"[0.0, 3837.334512, 6738.829137000001]",ALCISTA
3,ADES COCO 946ML 3PK,"[1597.535482, -952.95186, -134.11209000000008]",ALCISTA
4,ADES FRUT DNO 200 ML 40 PK,"[-56.35672, 0.0, 0.0]",BAJISTA
...,...,...,...
567,YOLI LIMON BOT 600L NR 12PK,"[162.20207899999514, 6393.279318000001, 3039.5...",ALCISTA
568,YOLI LIMON POSTMIX B-IN-B 9LT,"[2952.912959999998, 4039.9272960000017, 4630.5...",ALCISTA
569,YOLI LIMON V.RET. 1/2LT 24 PACK,"[786.1758719999998, 112.0089280000011, 1527.97...",ALCISTA
570,YOLI LIMON VRET 355ML 24 PACK,"[453.0, 639.0, 1881.0]",ALCISTA


In [10]:
tendencias = tendencias.reset_index(drop=True)
tendencias.columns = ['producto','tendencia','tipo']
tendencias

,producto,tendencia,tipo
0,(CCO+CCSA) 355ML LATA 24PK NR+4TRINEO,"[0.0, 0.0, 0.0]",ESTABLE
1,ADES ALMENDRA 946ML 3PK,"[2015.385986, -662.5664259999999, -85.29625600...",ALCISTA
2,ADES CHOCOLATE 200 ML 4PK,"[0.0, 3837.334512, 6738.829137000001]",ALCISTA
3,ADES COCO 946ML 3PK,"[1597.535482, -952.95186, -134.11209000000008]",ALCISTA
4,ADES FRUT DNO 200 ML 40 PK,"[-56.35672, 0.0, 0.0]",BAJISTA
...,...,...,...
567,YOLI LIMON BOT 600L NR 12PK,"[162.20207899999514, 6393.279318000001, 3039.5...",ALCISTA
568,YOLI LIMON POSTMIX B-IN-B 9LT,"[2952.912959999998, 4039.9272960000017, 4630.5...",ALCISTA
569,YOLI LIMON V.RET. 1/2LT 24 PACK,"[786.1758719999998, 112.0089280000011, 1527.97...",ALCISTA
570,YOLI LIMON VRET 355ML 24 PACK,"[453.0, 639.0, 1881.0]",ALCISTA


In [11]:
tendencias.to_csv('tendencias.csv')

In [12]:
df = pd.read_csv('input.csv',encoding='iso-8859-1')
df = df[['prod_A','prod_B','estado','canal','gec','lift','confidence', 'sales_a','sales_b','support_b']]

for column in ['prod_A','prod_B','estado','canal','gec']:
    df[column] = df[column].str.upper()
    
df = df.dropna()
df

,prod_A,prod_B,estado,canal,gec,lift,confidence,sales_a,sales_b,support_b
0,COCA COLA 600 ML 24 PK,CC 355ML NR PET 12PK,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.004435,0.973156,2.874746e+07,1.011189e+07,0.968859
1,COCA COLA 600 ML 24 PK,COCA COLA 3 LT PET NR 4 PK,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.006433,0.967023,2.874746e+07,2.351121e+07,0.960842
2,COCA COLA 600 ML 24 PK,COCA COLA 2.5 LT REF-PET 8 CJ,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.003575,0.945397,2.874746e+07,3.279004e+07,0.942028
3,COCA COLA 600 ML 24 PK,CC PET 1LTNR 12 PK,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.008326,0.949475,2.874746e+07,1.115415e+07,0.941635
4,COCA COLA 600 ML 24 PK,SIDRAL 600 ML PET NR 12 G,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.007613,0.945274,2.874746e+07,3.851277e+06,0.938132
...,...,...,...,...,...,...,...,...,...,...
2639247,CIEL 600 ML PET NR 12 PK,SIDRAL 355ML LATA LEAN NR 12PK,VDM ORIENTE,OTROS,PLATA,9.000000,1.000000,1.141223e+01,7.500000e-01,0.111111
2639248,CC PET 1LTNR 12 PK,SIDRAL 355ML LATA LEAN NR 12PK,VDM ORIENTE,OTROS,PLATA,0.000000,0.000000,1.056688e+02,7.500000e-01,0.111111
2639249,CIEL 600 ML PET NR 12 PK,DV EDGE 946ML 4 PK NR NARANJA NECTAR,VDM ORIENTE,OTROS,PLATA,9.000000,1.000000,1.141223e+01,6.664180e-01,0.111111
2639250,CC PET 1LTNR 12 PK,DV EDGE 946ML 4 PK NR NARANJA NECTAR,VDM ORIENTE,OTROS,PLATA,0.000000,0.000000,1.056688e+02,6.664180e-01,0.111111


In [13]:
df_sales = df[['prod_A','sales_a']]
df_sales.columns = ['product','sales']
df_sales = df_sales.groupby(by=["product"]).sum()
df_sales['category'] = 'NA'

df_sales.loc[(df_sales['sales'] < df_sales['sales'].quantile(.25)),'category'] = 'BAJA VENTA'
df_sales.loc[(df_sales['sales'] >= df_sales['sales'].quantile(.25)) & (df_sales['sales'] < df_sales['sales'].quantile(.5)),'category'] = 'MEDIA VENTA'
df_sales.loc[(df_sales['sales'] >= df_sales['sales'].quantile(.5)) & (df_sales['sales'] < df_sales['sales'].quantile(.75)),'category'] = 'ALTA VENTA'
df_sales.loc[(df_sales['sales'] >= df_sales['sales'].quantile(.75)),'category'] = 'MUY ALTA VENTA'

df_sales = df_sales.reset_index()
df_sales


,product,sales,category
0,1C LECHE UHT ENT 1L TRP NR 12P,6.013751e+07,MEDIA VENTA
1,ADES CHOCOLATE 200 ML 4PK,2.390912e+07,MEDIA VENTA
2,ADES FRUT DNO 946 ML 12 PK,3.252484e+07,MEDIA VENTA
3,ADES FRUT MGO 200 ML 4 PK,1.021864e+07,MEDIA VENTA
4,ADES FRUT MGO 946 ML 3 PK,9.669178e+07,ALTA VENTA
...,...,...,...
572,YOGURT PLÃ¡TANO 150 GR,7.143157e+03,BAJA VENTA
573,YOGURT PLÃ¡TANO 500 GR,8.571821e+03,BAJA VENTA
574,YOLI LIMON BOT 2L NR 8PK,6.450587e+08,MUY ALTA VENTA
575,YOLI LIMON BOT 600L NR 12PK,8.554939e+08,MUY ALTA VENTA


In [14]:
df_sales = df_sales.merge(tendencias, left_on='product', right_on='producto', how='inner')
del df_sales['producto']
df_sales

,product,sales,category,tendencia,tipo
0,ADES CHOCOLATE 200 ML 4PK,2.390912e+07,MEDIA VENTA,"[0.0, 3837.334512, 6738.829137000001]",ALCISTA
1,ADES FRUT DNO 946 ML 12 PK,3.252484e+07,MEDIA VENTA,"[1319.5076400000003, -859.6792200000004, 4548....",ALCISTA
2,ADES FRUT MGO 200 ML 4 PK,1.021864e+07,MEDIA VENTA,"[2375.43912, -740.810136, -1170.2489519999997]",ALCISTA
3,ADES FRUT MGO 946 ML 3 PK,9.669178e+07,ALTA VENTA,"[7044.215077999999, -8453.519119999999, 671.58...",BAJISTA
4,ADES FRUT MGO 946ML 12 PK,1.215638e+06,BAJA VENTA,"[-1403.4763080000002, -99.96269999999998, -39....",BAJISTA
...,...,...,...,...,...
371,VLLEFRU MZA BOT 600ML NR PET12P,1.631344e+08,ALTA VENTA,"[-920.3839990000015, -6490.885453999999, -4282...",BAJISTA
372,VLLEFRU UVA BOT 600M NR PET 12P,2.162924e+08,ALTA VENTA,"[2802.010312999999, -8858.288719999997, -4143....",BAJISTA
373,YOLI LIMON BOT 2L NR 8PK,6.450587e+08,MUY ALTA VENTA,"[2378.25274, 819.9899850000002, 1713.243679999...",ALCISTA
374,YOLI LIMON BOT 600L NR 12PK,8.554939e+08,MUY ALTA VENTA,"[162.20207899999514, 6393.279318000001, 3039.5...",ALCISTA


In [15]:
df =df.merge(df_sales[['product','category','tipo','tendencia']], left_on='prod_B',right_on='product', how='left')
df.columns = df.columns.str.replace('category', 'category_b')
del df['product']
df = df.dropna()
df

,prod_A,prod_B,estado,canal,gec,lift,confidence,sales_a,sales_b,support_b,category_b,tipo,tendencia
0,COCA COLA 600 ML 24 PK,CC 355ML NR PET 12PK,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.004435,0.973156,2.874746e+07,1.011189e+07,0.968859,MUY ALTA VENTA,ALCISTA,"[653824.9432499995, -113964.11999999918, 29578..."
1,COCA COLA 600 ML 24 PK,COCA COLA 3 LT PET NR 4 PK,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.006433,0.967023,2.874746e+07,2.351121e+07,0.960842,MUY ALTA VENTA,ALCISTA,"[-118601.60443199985, 167740.76649599895, 3747..."
2,COCA COLA 600 ML 24 PK,COCA COLA 2.5 LT REF-PET 8 CJ,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.003575,0.945397,2.874746e+07,3.279004e+07,0.942028,MUY ALTA VENTA,BAJISTA,"[1529131.292913001, -1384941.344742, -528984.2..."
3,COCA COLA 600 ML 24 PK,CC PET 1LTNR 12 PK,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.008326,0.949475,2.874746e+07,1.115415e+07,0.941635,MUY ALTA VENTA,ALCISTA,"[171274.69467200013, -32821.270308000036, 5239..."
4,COCA COLA 600 ML 24 PK,SIDRAL 600 ML PET NR 12 G,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.007613,0.945274,2.874746e+07,3.851277e+06,0.938132,MUY ALTA VENTA,BAJISTA,"[-13019.762717999984, -78676.821337, 90035.502..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486370,CIEL 600 ML PET NR 12 PK,CC 2.5L NO RET 8 PACK,VDM ORIENTE,OTROS,PLATA,0.000000,0.000000,1.141223e+01,1.408918e+01,0.111111,MUY ALTA VENTA,ALCISTA,"[-173073.19889699994, -84738.02847100003, 1322..."
2486371,CC PET 1LTNR 12 PK,CC 2.5L NO RET 8 PACK,VDM ORIENTE,OTROS,PLATA,0.000000,0.000000,1.056688e+02,1.408918e+01,0.111111,MUY ALTA VENTA,ALCISTA,"[-173073.19889699994, -84738.02847100003, 1322..."
2486379,CIEL 600 ML PET NR 12 PK,DV EDGE 946ML 4 PK NR NARANJA NECTAR,VDM ORIENTE,OTROS,PLATA,9.000000,1.000000,1.141223e+01,6.664180e-01,0.111111,ALTA VENTA,ALCISTA,"[1227.5419559999991, -775.7105519999986, 938.9..."
2486380,CC PET 1LTNR 12 PK,DV EDGE 946ML 4 PK NR NARANJA NECTAR,VDM ORIENTE,OTROS,PLATA,0.000000,0.000000,1.056688e+02,6.664180e-01,0.111111,ALTA VENTA,ALCISTA,"[1227.5419559999991, -775.7105519999986, 938.9..."


In [16]:
df.to_csv('metrics.csv', index=False)
df_sales.to_csv('sales.csv', index=False)